In [37]:
# [Celda 1] - Importación de librerías
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import seaborn as sns
import matplotlib.pyplot as plt
import joblib
import optuna
from sklearn.model_selection import cross_val_score, train_test_split


In [38]:
# [Celda 2] - Carga de datos
# Leemos el dataset
df = pd.read_csv('youtoxic_english_1000.csv')

# Mostramos las primeras filas y la información del dataset
print("Primeras filas del dataset:")
display(df.head())
print("\nInformación del dataset:")
display(df.info())

Primeras filas del dataset:


,CommentId,VideoId,Text,IsToxic,IsAbusive,IsThreat,IsProvocative,IsObscene,IsHatespeech,IsRacist,IsNationalist,IsSexist,IsHomophobic,IsReligiousHate,IsRadicalism
0,Ugg2KwwX0V8-aXgCoAEC,04kJtp6pVXI,If only people would just take a step back and...,False,False,False,False,False,False,False,False,False,False,False,False
1,Ugg2s5AzSPioEXgCoAEC,04kJtp6pVXI,Law enforcement is not trained to shoot to app...,True,True,False,False,False,False,False,False,False,False,False,False
2,Ugg3dWTOxryFfHgCoAEC,04kJtp6pVXI,\nDont you reckon them 'black lives matter' ba...,True,True,False,False,True,False,False,False,False,False,False,False
3,Ugg7Gd006w1MPngCoAEC,04kJtp6pVXI,There are a very large number of people who do...,False,False,False,False,False,False,False,False,False,False,False,False
4,Ugg8FfTbbNF8IngCoAEC,04kJtp6pVXI,"The Arab dude is absolutely right, he should h...",False,False,False,False,False,False,False,False,False,False,False,False



Información del dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   CommentId        1000 non-null   object
 1   VideoId          1000 non-null   object
 2   Text             1000 non-null   object
 3   IsToxic          1000 non-null   bool  
 4   IsAbusive        1000 non-null   bool  
 5   IsThreat         1000 non-null   bool  
 6   IsProvocative    1000 non-null   bool  
 7   IsObscene        1000 non-null   bool  
 8   IsHatespeech     1000 non-null   bool  
 9   IsRacist         1000 non-null   bool  
 10  IsNationalist    1000 non-null   bool  
 11  IsSexist         1000 non-null   bool  
 12  IsHomophobic     1000 non-null   bool  
 13  IsReligiousHate  1000 non-null   bool  
 14  IsRadicalism     1000 non-null   bool  
dtypes: bool(12), object(3)
memory usage: 35.3+ KB


None

In [39]:
# [Celda 3] - Preparación y limpieza de datos

# Primero mostramos información sobre valores nulos en el dataset
print("Valores nulos por columna antes de la limpieza:")
print(df.isnull().sum())
print("\nTotal de filas antes de la limpieza:", len(df))

# Definimos las columnas que indican contenido dañino
toxic_columns = ['IsToxic', 'IsAbusive', 'IsThreat', 'IsProvocative', 
                'IsObscene', 'IsHatespeech', 'IsRacist', 'IsNationalist',
                'IsSexist', 'IsHomophobic', 'IsReligiousHate', 'IsRadicalism']

# Convertir los valores "TRUE"/"FALSE" a 1/0 en las columnas tóxicas (si es necesario)
df[toxic_columns] = df[toxic_columns].replace({'TRUE': 1, 'FALSE': 0})

# Creamos una nueva columna que será 1 si cualquiera de las categorías es 1
df['is_harmful'] = df[toxic_columns].any(axis=1).astype(int)

# Preparamos X (textos) e y (etiquetas)
X = df['Text']
y = df['is_harmful']

# Mostramos estadísticas finales
print("\nEstadísticas finales:")
print("-" * 50)
print("Distribución de mensajes dañinos vs no dañinos:")
print(y.value_counts(normalize=True))
print("\nDistribución por tipo de contenido dañino:")
for col in toxic_columns:
    positivos = df[col].sum()
    porcentaje = (positivos / len(df)) * 100
    print(f"{col}: {positivos} casos ({porcentaje:.2f}%)")


Valores nulos por columna antes de la limpieza:
CommentId          0
VideoId            0
Text               0
IsToxic            0
IsAbusive          0
IsThreat           0
IsProvocative      0
IsObscene          0
IsHatespeech       0
IsRacist           0
IsNationalist      0
IsSexist           0
IsHomophobic       0
IsReligiousHate    0
IsRadicalism       0
dtype: int64

Total de filas antes de la limpieza: 1000

Estadísticas finales:
--------------------------------------------------
Distribución de mensajes dañinos vs no dañinos:
is_harmful
0    0.538
1    0.462
Name: proportion, dtype: float64

Distribución por tipo de contenido dañino:
IsToxic: 462 casos (46.20%)
IsAbusive: 353 casos (35.30%)
IsThreat: 21 casos (2.10%)
IsProvocative: 161 casos (16.10%)
IsObscene: 100 casos (10.00%)
IsHatespeech: 138 casos (13.80%)
IsRacist: 125 casos (12.50%)
IsNationalist: 8 casos (0.80%)
IsSexist: 1 casos (0.10%)
IsHomophobic: 0 casos (0.00%)
IsReligiousHate: 12 casos (1.20%)
IsRadicalism: 0 c

In [40]:
# [Celda 4] - División de datos
# Dividimos los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2,
    random_state=42,
    stratify=y  # Mantenemos la proporción de clases
)

print("Tamaño del conjunto de entrenamiento:", len(X_train))
print("Tamaño del conjunto de prueba:", len(X_test))

Tamaño del conjunto de entrenamiento: 800
Tamaño del conjunto de prueba: 200


In [41]:
# [Celda 5] - Vectorización del texto
# Convertimos el texto a una matriz de características TF-IDF
vectorizer = TfidfVectorizer(
    max_features=5000,  # Limitamos el número de características
    min_df=2,          # Ignoramos términos que aparecen en menos de 2 documentos
    stop_words='english'  # Removemos stopwords en inglés
)

# Transformamos los textos
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

print("Dimensiones de la matriz de características de entrenamiento:", X_train_vectorized.shape)

Dimensiones de la matriz de características de entrenamiento: (800, 1496)


In [42]:
# Definir la función objetivo para Optuna
def objective(trial):
    # Definir los hiperparámetros a optimizar
    C = trial.suggest_loguniform('C', 1e-3, 1e3)  # Regularización (log escala para explorar valores pequeños)
    kernel = trial.suggest_categorical('kernel', ['linear', 'rbf', 'poly', 'sigmoid'])  # Tipo de kernel
    gamma = trial.suggest_loguniform('gamma', 1e-4, 1e1) if kernel != 'linear' else 'scale'  # Coeficiente gamma
    class_weight = trial.suggest_categorical('class_weight', ['balanced', None])  # Manejo de clases desbalanceadas
    tol = trial.suggest_loguniform('tol', 1e-5, 1e-1)  # Tolerancia para criterio de parada
    degree = trial.suggest_int('degree', 2, 5) if kernel == 'poly' else 3  # Grado del polinomio (solo para poly)
    coef0 = trial.suggest_uniform('coef0', 0, 1) if kernel in ['poly', 'sigmoid'] else 0.0  # Coeficiente independiente
    shrinking = trial.suggest_categorical('shrinking', [True, False])  # Activar o no shrinking
    max_iter = trial.suggest_int('max_iter', -1, 10000)  # Máximo número de iteraciones (-1 para sin límite)
    
    # Crear y entrenar el modelo SVM con los hiperparámetros propuestos
    svm_model = SVC(
        C=C,
        kernel=kernel,
        gamma=gamma,
        probability=True,
        random_state=42,
        class_weight=class_weight,
        tol=tol,
        degree=degree,
        coef0=coef0,
        shrinking=shrinking,
        max_iter=max_iter
    )
    
    # Evaluar el modelo con validación cruzada
    scores = cross_val_score(svm_model, X_train_vectorized, y_train, cv=3, scoring='accuracy')
    accuracy = scores.mean()  # Promedio de accuracy
    
    return accuracy  # Optuna maximiza este valor

# Crear un estudio de optimización
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=200)  # Ajusta el número de pruebas según el tiempo disponible

# Resultados de la mejor prueba
print("Best trial:")
print("  Value: ", study.best_trial.value)
print("  Params: ")
for key, value in study.best_trial.params.items():
    print(f"    {key}: {value}")


[I 2024-11-07 12:45:38,828] A new study created in memory with name: no-name-b2fb581b-c8fa-413a-a2ce-d67564a37997
C:\Users\Administrator\AppData\Local\Temp\ipykernel_3152\2276161676.py:9: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  C = trial.suggest_loguniform('C', 1e-3, 1e3)  # Regularización (log escala para explorar valores pequeños)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_3152\2276161676.py:11: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  gamma = trial.suggest_loguniform('gamma', 1e-4, 1e1) if kernel != 'linear' else 'scale'  # Coeficiente gamma
C:\Users\Administrator\AppData\Local\Temp\ipykernel_3152\2276161676.py:13: FutureWarning: suggest_logunifor

Best trial:
  Value:  0.698778782161396
  Params: 
    C: 109.42844096035671
    kernel: rbf
    gamma: 1.1662316858478285
    class_weight: None
    tol: 0.00043366566542372227
    shrinking: False
    max_iter: 1697


In [43]:
best_params = study.best_trial.params
print("Mejores hiperparámetros encontrados:", best_params)

Mejores hiperparámetros encontrados: {'C': 109.42844096035671, 'kernel': 'rbf', 'gamma': 1.1662316858478285, 'class_weight': None, 'tol': 0.00043366566542372227, 'shrinking': False, 'max_iter': 1697}


In [44]:
# Extraer los mejores parámetros
C = best_params.get('C')
kernel = best_params.get('kernel')
gamma = best_params.get('gamma') if kernel != 'linear' else 'scale'
class_weight = best_params.get('class_weight')
tol = best_params.get('tol')

# Crear el modelo con los mejores hiperparámetros
final_svm_model = SVC(
    C=C,
    kernel=kernel,
    gamma=gamma,
    probability=True,
    random_state=42,
    class_weight=class_weight,
    tol=tol
)

# Entrenar el modelo con el conjunto de entrenamiento completo
final_svm_model.fit(X_train_vectorized, y_train)

print("Modelo final entrenado con los mejores hiperparámetros!")

Modelo final entrenado con los mejores hiperparámetros!


In [52]:
# Evaluar el modelo en el conjunto de prueba
from sklearn.metrics import accuracy_score, classification_report

# Predicciones de probabilidades para las clases
y_prob = final_svm_model.predict_proba(X_test_vectorized)[:, 1]  # Probabilidad de la clase positiva (por ejemplo, clase 1)

# Ajuste del umbral de decisión
threshold = 0.46  # Cambia este valor según lo que desees probar
y_pred = (y_prob >= threshold).astype(int)  # Predicciones basadas en el umbral

# Métricas de rendimiento
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Accuracy en conjunto de prueba (con umbral {threshold}):", accuracy)
print("Reporte de clasificación:\n", report)

Accuracy en conjunto de prueba (con umbral 0.46): 0.72
Reporte de clasificación:
               precision    recall  f1-score   support

           0       0.72      0.79      0.75       108
           1       0.72      0.64      0.68        92

    accuracy                           0.72       200
   macro avg       0.72      0.71      0.72       200
weighted avg       0.72      0.72      0.72       200



In [58]:
# Guardar el modelo en un archivo
joblib.dump(final_svm_model, 'svm_model.joblib')

print("Modelo guardado exitosamente en 'svm_model.joblib'")

Modelo guardado exitosamente en 'svm_model.joblib'


In [55]:
# [Celda 8 corregida] - Función para detectar nuevos mensajes
def detect_hate_speech(text):
    """
    Analiza un nuevo texto para detectar si contiene mensaje de odio
    """
    # Vectorizamos el texto
    text_vectorized = vectorizer.transform([text])
    
    # Obtenemos la predicción y la probabilidad
    # No necesitamos envolver text_vectorized en otra lista
    is_harmful = final_svm_model.predict(text_vectorized)[0]
    prob_harmful = final_svm_model.predict_proba(text_vectorized)[0][1]
    
    return {
        'texto': text,
        'es_dañino': bool(is_harmful),
        'probabilidad': float(prob_harmful)
    }

In [56]:
# [Celda 9 corregida] - Ejemplo de uso
# Probamos el modelo con algunos ejemplos
textos_prueba = [
    "This is a stupid message about kills",
    "I love spending time with my family"
]

for texto in textos_prueba:
    resultado = detect_hate_speech(texto)
    print(f"\nTexto: {resultado['texto']}")
    print(f"Es dañino: {resultado['es_dañino']}")
    print(f"Probabilidad: {resultado['probabilidad']:.2f}")


Texto: This is a stupid message about kills
Es dañino: True
Probabilidad: 0.77

Texto: I love spending time with my family
Es dañino: False
Probabilidad: 0.13
